# Error Handling in JavaScript  

## Introduction  

Error handling ensures that when something goes wrong in code, the application does not crash abruptly and the user still gets a meaningful response or message.[1][2]
In JavaScript, error handling focuses on detecting events that disturb the normal flow of execution and responding to them gracefully using constructs like `try`, `catch`, `finally`, and `throw`.[3][1]

***

## What Is an Error?  

An **error** is an event that disrupts or changes the expected, normal flow of program execution.[2]
When a program is written, the developer assumes a particular sequence of statements will execute, and any issue that prevents this expected sequence from running correctly is considered an error.[4][2]

### Normal Flow vs Error  

- **Normal flow**: All lines execute in the intended sequence without interruption.  
- **Error scenario**: At some line, something unexpected happens (like using an undeclared variable), execution stops there, and the rest of the code does not run.  

The goal of error handling is not to eliminate all errors (which is impossible in real systems) but to **handle them gracefully** so that:  
- Users get informative feedback instead of a hanging page.  
- The system remains stable and predictable.  

***

## Types of Errors  

The lecture broadly divides errors into two categories: **compile-time errors** and **runtime errors**.[5][4]

### Compile-Time Errors  

Compile-time errors are detected **before** the program actually runs, typically when the JavaScript engine parses the code.[4][5]

Key points:  
- Occur during **parsing** or initial processing of the script.  
- Usually caused by **syntax mistakes** or invalid structure.  
- The code does not start executing until these are fixed.  

Example concept:  
- Writing `consol.log("Hello")` instead of `console.log("Hello")` or missing a closing parenthesis leads to a **SyntaxError**, which is caught before the code executes.[1][4]

### Runtime Errors  

Runtime errors occur **while the code is executing**.[5][4]

Key points:  
- The script starts running successfully, but at some statement an issue appears.  
- Often caused by operations on invalid data or references, not pure syntax.  
- The error stops further execution from that point unless handled.  

Example concept:  
- Calling `console.log(x)` when `x` is not declared produces a **ReferenceError**, which appears only when the line executes.[2][1]

***

## Why Error Handling Matters  

Errors appear in every non-trivial system regardless of how good the developers are.[2]
What differentiates good systems is **how gracefully they handle errors**, especially from the user’s perspective.[2]

### User Experience Example  

Consider a page that loads tutorials from a server:  
- Bad handling: The loader/spinner keeps rotating forever if the server has an issue. The user waits indefinitely and gets no explanation.  
- Good handling: After a timeout, the page stops trying, shows a clear message like “Our servers are facing an issue; please try again later,” maybe with an estimated recovery time.  

Good error handling:  
- Saves user time.  
- Builds trust by explaining what went wrong.  
- Makes debugging and maintenance easier.  

***

## The try…catch Block  

JavaScript uses the `try…catch` construct as the primary mechanism for handling exceptions.[6][1]

### Basic Structure  

```js
try {
  // code that may throw an error
} catch (error) {
  // code to handle the error
}
```

- The `try` block contains code that **might** throw an error.  
- If an error occurs inside `try`, JavaScript **jumps immediately** to the `catch` block.  
- Lines in `try` **after the error line do not execute** once an error is thrown.[6][1]

### Flow of Control  

1. Execution enters the `try` block.  
2. Each line in `try` runs in order until:  
   - All lines succeed (no error), or  
   - A line throws an error:  
     - Execution in `try` stops at the error line.  
     - Control jumps to `catch`.  
3. Inside `catch`, the parameter (often named `e` or `error`) holds the error object, which can be logged, transformed, or used to trigger fallback logic.[1][6]

Example pattern:  
```js
try {
  console.log("Try block starts");
  console.log(x); // x not defined → ReferenceError
  console.log("This will never run");
} catch (err) {
  console.log("Inside catch block");
  console.log("Your error is here:", err);
}
```

Key observations:  
- The line after the failing statement in `try` is skipped.  
- The `catch` block runs once, and any logic inside it executes normally.  

### What To Do Inside catch  

Typical operations inside `catch` include:[1][2]
- Retrying an operation (e.g., making a request again).  
- Using a **fallback mechanism** (e.g., show cached data instead of live API data).  
- Throwing a **custom error** for higher-level code.  
- Logging error details to the console or a backend logging system.  

***

## The finally Block  

The `finally` block defines code that runs **every time**, regardless of whether an error occurred or not.[3][6][1]

### Structure  

```js
try {
  // risky code
} catch (error) {
  // handle error
} finally {
  // always runs
}
```

Key characteristics:  
- Runs after `try` and (if present) `catch`.  
- Executes whether:  
  - No error occurred.  
  - An error was thrown and caught.  
- Often used for **cleanup** tasks, such as closing connections, clearing timers, stopping loaders, or releasing resources.[7][1]

### Flow with Error  

- Error in `try` → `catch` executes → `finally` executes → flow continues.  

### Flow without Error  

- No error in `try` → `catch` is skipped → `finally` executes → flow continues.  

Example pattern:  
```js
try {
  console.log("Try block starts");
  console.log(x); // ReferenceError
} catch (error) {
  console.log("Inside catch");
} finally {
  console.log("I will run every time");
}
```

This ensures that certain actions (like hiding a loader) always happen, no matter what went wrong in the code.  

***

## Error Object and Stack Trace  

In the `catch(error)` block, the parameter (`error`) is typically an **Error object**.[6][1]

Key features:  
- Contains a **message** describing the issue.  
- Often contains a **stack trace** showing where the error originated and how the call stack led there.  

Logging `error` (or `error.message`) helps developers quickly identify the location and cause of errors during debugging.[1][2]

***

## Creating and Throwing Custom Errors with throw  

Sometimes the low-level error (e.g., an HTTP status code or a numeric error code) is not meaningful for the end user, so you want to **convert it into a more descriptive, custom error**.[8][2][1]

### throw Keyword  

The `throw` statement is used to **manually generate** an error.[3][1]

Common usage with the built-in `Error` constructor:  
```js
throw new Error("Custom error message");
```

Effects:  
- Immediately stops normal execution at the `throw` line.  
- Control jumps to the nearest enclosing `catch` block (if present).  
- If not caught, the error propagates up and can terminate the script.  

### Realistic Scenario: API Error Codes  

Imagine a company’s system talks to a government API that returns error codes like `401`, `402`, `403`, etc., each with a technical meaning.[8][2]

Example mapping idea:  
- `401`: Name field invalid or blank.  
- `402`: Amount value above allowed limit.  
- `403`: Contains invalid characters or symbols.  

Directly showing “Error 401” to users is confusing.  
Instead, the application can **interpret** the code and throw a user-friendly error:  

```js
let errorCode = 401;

if (errorCode === 401) {
  throw new Error("Error in name field");
} else if (errorCode === 402) {
  throw new Error("Amount must be below allowed limit");
} else if (errorCode === 403) {
  throw new Error("Input contains invalid characters");
}
```

This way:  
- The system respects the underlying API error structure.  
- The user sees a clear, understandable message.  

### Combining try…catch with throw  

A common pattern is:  
- Run risky code in `try`.  
- If a low-level error appears (like an undefined variable or an invalid response), handle it or transform it in `catch`.  
- Optionally `throw` a **new, higher-level error** that is more meaningful for calling code.  

Simplified pattern:  
```js
try {
  // risky code that may fail
} catch (err) {
  // log or inspect original error
  throw new Error("Please check your input before proceeding");
}
```

***

## Putting It All Together: Flow Cases  

### Case 1: Error Occurs  

- Enter `try`.  
- Run statements until one throws an error.  
- Skip remaining lines in `try`.  
- Jump into `catch`, run its logic.  
- Execute `finally` (if present).  
- Continue with code after the whole construct.  

### Case 2: No Error  

- Enter `try`.  
- All statements in `try` run successfully.  
- Skip `catch` (because no error).  
- Execute `finally` (if present).  
- Continue with code after the whole construct.  

In both cases, `finally` gives a reliable place for “always-do-this” logic.  

***

## Key Patterns and Best Practices  

- **Always handle errors**: Do not ignore exceptions; log them or show helpful messages.[2]
- **Use try…catch around risky sections**: For example, network calls, JSON parsing, or user input processing.[1][2]
- **Use finally for cleanup**: Stop spinners, close connections, or free resources regardless of success or failure.[7][1]
- **Use custom errors for clarity**: Map low-level codes (like API status or numeric codes) to descriptive, user-friendly messages using `throw new Error("message")`.[8][2]
- **Do not overuse try…catch**: Wrap logical groups of risky operations, not the entire application.  

***

## Summary of Main Takeaways  

- An **error** is any event that disrupts the normal, expected flow of code execution.  
- Errors are broadly categorized into **compile-time errors** (detected during parsing, usually syntax-related) and **runtime errors** (detected while executing code, such as using undeclared variables).  
- The `try` block encloses code that may fail, the `catch` block handles any resulting error, and the `finally` block runs in all cases, often for cleanup.  
- The `throw` keyword allows creation of **custom, descriptive errors**, which is especially useful when converting technical error codes (like API response codes) into meaningful user messages.  
- Good JavaScript error handling is about **graceful degradation**: keeping the app stable, providing clear feedback, and maintaining a good user experience even when things go wrong.